In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from numpy.random import default_rng
from collections import Counter
import copy
import matplotlib.pyplot as plt
# from itertools import product

## hexadecimal generator

In [10]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [11]:
## world size
world_size_x = 128
world_size_y = 128

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)


In [12]:
individuals_sum_dup = sum_duplicated_neurons(individuals)

## initial brain and position generator

In [13]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## brain generator
for individual in individuals_sum_dup:
    
    ## init brain neuron generator
    print(individual)
    ## preprocessing
    edges = individuals_sum_dup[individual]
    edges = [tuple(edges[i]) for i in edges]
    edges_no_weight = [i[:-1] for i in edges]
    
    G = nx.MultiDiGraph()
    G.add_edges_from(edges)
    
    out_list = list(set([i[1] for i in edges if 'out' in i[1]]))
    init_list = list(set([i[0] for i in edges if 'in' in i[0]]))

    # generate dict of paths
    dic_of_paths = generate_dict_of_paths(out_list, init_list, G)

    lis = filtered_neurons_paths(dic_of_paths)
    
    dic = append_weight_to_neurons_in_path(lis, edges_no_weight, edges)
    print(dic)
    
        
        

#     print(dic)
    print()

0
{'out2': {'mid0': -0.8027080636332302, 'mid1': {'in2': -1.477674347627041, 'in1': 0.8046390965969785, 'mid0': 2.1775675263238212}, 'out2': {'mid1': -1.0157179917594994}}, 'out1': {'out1': {'in1': 0.36282618647947507}}, 'out0': {'mid0': -0.8027080636332302, 'mid1': {'in2': -1.477674347627041, 'in1': 0.8046390965969785, 'mid0': 2.1775675263238212}, 'mid2': {'in1': -1.2309476575614222, 'mid0': 3.6696474896993743, 'mid1': -3.249565084694033}, 'out0': {'mid1': 1.1674652830764536, 'in1': -2.353486952540821, 'mid2': 1.6059819929803143, 'in0': 2.8919884022585074, 'mid0': 1.0667480543262629}}}

1
{'out2': {'mid2': -0.985832393806578, 'out2': {'mid2': 3.661101785441782}}, 'out1': {'mid0': {'in2': -1.3317869678010072, 'in1': -0.9715244925988097, 'mid2': 2.5182969632229515, 'mid0': -3.9030672974210288}, 'mid2': -0.985832393806578, 'out1': {'in1': -0.023561727453074926, 'mid2': 2.3359072180680602, 'mid0': -0.31521440561574854, 'in2': -1.109842820082405}}, 'out0': {'mid0': {'in1': -0.9715244925988

In [18]:
def out_to_weight(dic):
    '''weight for nested input ie.  0mid:{1mid} or out:{0mid, 0in}
    use 'mid' neuron only if feeded with 'in' neurons'''
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key != key:
                    dic[key][sub_key] += dic[sub_key]

            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key in dic:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)

In [20]:
out_to_weight(dic['out0'])
remove_mid_from_dict(dic['out0'])

TypeError: unsupported operand type(s) for +=: 'float' and 'dict'

In [41]:
d = {'mid0': -0.9968891785861455,
 'mid1': -0.4076410405214311,
 'out0': 0.9588224118999349}

In [43]:
def remove_mid_from_dict(dic):
    mid_list = [i for i in dic if 'mid' in i]
    for mid in mid_list:
        dic.pop(mid)

remove_mid_from_dict(d)    

In [44]:
d

{'out0': 0.9588224118999349}

In [8]:
initial_dic_of_output_neurons = {}

## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
    
    ## init brain neuron generator
    
    ## preprocessing
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    
    initial_dic_of_output_neurons[individual] = {}
    initial_dic_of_output_neurons[individual]['output'] = dic
    
    ## position
#     initial_dic_of_output_neurons[individual]['position'] = {}
#     initial_dic_of_output_neurons[individual]['position']['x'] = x[individual]
#     initial_dic_of_output_neurons[individual]['position']['y'] = y[individual] 
    
    ## position history
    initial_dic_of_output_neurons[individual]['history_position'] = [(x[individual], y[individual])]

In [9]:
initial_dic_of_output_neurons

{0: {'output': {'out0': -0.9941392171560546,
   'out1': -0.3995677412004998,
   'out2': -0.559667317504093},
  'history_position': [(90, 1)]},
 1: {'output': {'out0': -0.07093210467435282,
   'out1': -0.4384494215569999,
   'out2': 0.5525715141627122},
  'history_position': [(23, 123)]},
 2: {'output': {'out0': 0.9984096476650253,
   'out1': 0.8128864397791226,
   'out2': 0.9895769484375675},
  'history_position': [(119, 73)]},
 3: {'output': {'out1': -0.9999982503498028, 'out2': 0.9998047532689923},
  'history_position': [(121, 93)]}}

In [10]:
a = [(3, 0),(4,1),(5,2)]

In [11]:
def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

print(slope_intercept(3,0,2,1))

(-1.0, 3.0)


In [12]:
a

[(3, 0), (4, 1), (5, 2)]

In [13]:
if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
print(x)

3


In [14]:
a[-2][0]

4